In [1]:
import sys
import os
import pandas as pd
import numpy as np

from github import Github, Repository
from GithubCredentials import my_credentials
from enum import Enum

sys.path.append(os.path.join(os.path.dirname(sys.path[0]),'harpcrawler'))
from harprepo.device import DeviceRepo, TemplateDeviceRepo
from harprepo.peripheral import PeripheralRepo, TemplatePeripheralRepo


In [2]:
## Setup
gh = Github(login_or_token=my_credentials())


In [3]:
## Grab the template repositories
device_template = TemplateDeviceRepo(gh.get_repo("bruno-f-cruz/device.template"))
peripheral_template = TemplatePeripheralRepo(gh.get_repo("bruno-f-cruz/peripheral.template"))

In [4]:
harp_organization = gh.get_organization("harp-tech")

## Get all "Device.*" repositories
device_diagnosis = pd.DataFrame(columns=device_template.filetree)
device_diagnosis = device_diagnosis.astype('bool')

device_repos = [DeviceRepo(repo, device_template)\
    for repo in harp_organization.get_repos()\
        if (("device." in repo.full_name) and not("template" in repo.full_name))]

for device_repo in device_repos:
    device_diagnosis = pd.concat([device_diagnosis,\
        pd.DataFrame(device_repo.exist_harpfiles(), index = [device_repo.repository.full_name.split("/")[-1]])\
            ], axis=0, ignore_index=False)


## Write to google sheets:
import gspread
gc = gspread.service_account()
sh = gc.open("GitHarpCrawler")

cols = device_diagnosis.columns.to_list()
cols.insert(0,'Device')
device_diagnosis['Device'] = device_diagnosis.index
device_diagnosis = device_diagnosis[cols]
device_diagnosis = device_diagnosis.replace({True: '\u2705', False: '\u274C'})
print(device_diagnosis)

try:
    worksheet = sh.add_worksheet(title="devices", rows = 100, cols= 100)
except:
    sh.del_worksheet(sh.worksheet(title="devices"))
    worksheet = sh.add_worksheet(title="devices", rows = 100, cols= 100)
worksheet.update([device_diagnosis.columns.values.tolist()] + device_diagnosis.values.tolist())



                                                  Device .gitignore LICENSE  \
device.rfidreader                      device.rfidreader          ❌       ❌   
device.behavior-before            device.behavior-before          ✅       ❌   
device.behavior                          device.behavior          ✅       ❌   
device.audioswitch                    device.audioswitch          ✅       ❌   
device.cameratriggercontrol  device.cameratriggercontrol          ❌       ❌   
device.clocksync                        device.clocksync          ❌       ❌   
device.loadcells                        device.loadcells          ✅       ❌   
device.rgbarray                          device.rgbarray          ✅       ❌   
device.multipwm                          device.multipwm          ❌       ❌   
device.synchronizer                  device.synchronizer          ✅       ❌   
device.ledarray                          device.ledarray          ✅       ❌   

                            README.md Assets/pcb.pn

{'spreadsheetId': '1ZjgfpKjjQnCMx7Z6JxweBzrvd6ZNkaRnnd75J6-5pSM',
 'updatedRange': 'devices!A1:J12',
 'updatedRows': 12,
 'updatedColumns': 10,
 'updatedCells': 120}

In [5]:
harp_organization = gh.get_organization("harp-tech")

## Get all "Peripherals.*" repositories
peripherals_diagnosis = pd.DataFrame(columns=peripheral_template.filetree)
peripherals_diagnosis = peripherals_diagnosis.astype('bool')

peripheral_repos = [PeripheralRepo(repo, peripheral_template)\
    for repo in harp_organization.get_repos()\
        if (("peripheral." in repo.full_name) and not("template" in repo.full_name))]

for peripheral_repo in peripheral_repos:
    peripherals_diagnosis = pd.concat([peripherals_diagnosis,\
        pd.DataFrame(peripheral_repo.exist_harpfiles(), index = [peripheral_repo.repository.full_name.split("/")[-1]])\
            ], axis=0, ignore_index=False)


## Write to google sheets:
import gspread
gc = gspread.service_account()
sh = gc.open("GitHarpCrawler")

cols = peripherals_diagnosis.columns.to_list()
cols.insert(0,'Peripheral')
peripherals_diagnosis['Peripheral'] = peripherals_diagnosis.index
peripherals_diagnosis = peripherals_diagnosis[cols]
peripherals_diagnosis = peripherals_diagnosis.replace({True: '\u2705', False: '\u274C'})

try:
    worksheet = sh.add_worksheet(title="peripherals", rows = 100, cols= 100)
except:
    sh.del_worksheet(sh.worksheet(title="peripherals"))
    worksheet = sh.add_worksheet(title="peripherals", rows = 100, cols= 100)
worksheet.update([peripherals_diagnosis.columns.values.tolist()] + peripherals_diagnosis.values.tolist())

In [6]:
worksheet.update([peripherals_diagnosis.columns.values.tolist()] + peripherals_diagnosis.values.tolist())

{'spreadsheetId': '1ZjgfpKjjQnCMx7Z6JxweBzrvd6ZNkaRnnd75J6-5pSM',
 'updatedRange': 'peripherals!A1:G8',
 'updatedRows': 8,
 'updatedColumns': 7,
 'updatedCells': 56}